In [2]:
# I would consider 3 options based on my expertise and the techs applied in the project. 

# option A
import pandas as pd
import duckdb
# or 
# import pandasql

# option B
# import polars as pl
# and use query() method

# option C (if you already have the spark env installed)
# from pyspark.sql import SparkSession
# and use sql() method, previous creating a tempView of your df

In [3]:
# coupons table import
coupons = pd.read_csv("coupons.csv", sep=",", header=0)

In [4]:
duckdb.query(''' 
        select * from coupons
             '''
        ).df()

,id,coupon_code
0,1,COUPON123
1,2,save20
2,3,DISCOUNT50


In [16]:
# orders table import
orders = pd.read_csv('orders.csv', 
                     sep=',', 
                     header=0, 
                     on_bad_lines='skip')
orders

,coupon_id,dt,total_amount
0,1.0,2023-04-29 13:55:09,86.17
1,1.0,2023-08-16 07:33:14,76.09
2,2.0,2022-10-14 00:49:14,43.61
3,2.0,2022-10-18 03:52:48,33.06
4,2.0,2022-10-19 10:23:47,17.46
5,2.0,2023-01-19 15:53:00,95.16
6,2.0,2023-01-20 05:40:15,97.40
7,2.0,2023-04-12 13:43:46,75.47
8,2.0,2023-04-19 11:05:46,28.17
9,3.0,2022-08-28 23:53:02,79.72


In [21]:
# handle the null values and parse the order column to int
orders['coupon_id'] = orders['coupon_id'].fillna(value=0)
orders['coupon_id'] = orders['coupon_id'].astype('int')
orders

,coupon_id,dt,total_amount
0,1,2023-04-29 13:55:09,86.17
1,1,2023-08-16 07:33:14,76.09
2,2,2022-10-14 00:49:14,43.61
3,2,2022-10-18 03:52:48,33.06
4,2,2022-10-19 10:23:47,17.46
5,2,2023-01-19 15:53:00,95.16
6,2,2023-01-20 05:40:15,97.40
7,2,2023-04-12 13:43:46,75.47
8,2,2023-04-19 11:05:46,28.17
9,3,2022-08-28 23:53:02,79.72


In [34]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   coupon_id     20 non-null     int32  
 1   dt            20 non-null     object 
 2   total_amount  20 non-null     float64
dtypes: float64(1), int32(1), object(1)
memory usage: 528.0+ bytes


### Online Store Coupon Analysis

<p>An online store is interested in analyzing the efficiency of their coupon codes. They want to understand the sales performance of each coupon over time.</p>
<p>The result should contain the following columns: coupon_code | month year | monthly_sales | mom growth | usage_rank | avg_last_3_months.</p>
<ul>
    <li>coupon code - unique code of the coupon</li>
    <li>month year - month and year of the sales data for a specific coupon monthly_sales - total sales amount for the month for a specific coupon
    <li>mom growth-Month-over-Month growth rate in sales for a specific coupon
    <li>usage_rank - rank of the coupon based on its monthly sales for that month
    <li>avg_last_3_months - average sales amount for the last 3 months for the coupon, including the current month, rounded to two decimal places
</ul>
<p>The result should be sorted in ascending order by month year, then in ascending order by usage_rank.</p>
<p>Note:</p>
<ul>
    <li>Month-over-Month growth is calculated as ((Current Month Sales Previous Month Sales) / Previous Month Sales) * 100%.</li>
    <li>If there is no previous month's sales data for a coupon, display Month-over-Month growth as 'N/A'.</li>
</ul>
<br>
<img src="expected_output.png" style="display=block; margin:auto; width: 40%;">

### Resolution

In [144]:
duckdb.query(''' 
        -- mother table, considering date as string
        -- if it's a time stamp we could parse it to date, and then concat year() and month().
        WITH monthly_table AS (
            SELECT
                coupon_code,
                LEFT(ord.dt, 7) AS year_month, 
                SUM(total_amount) AS monthly_sales
            FROM 
                orders ord
            -- just coupon cases, not orders without them; hence inner join
            INNER JOIN
                coupons cup
                ON ord.coupon_id = cup.id
            GROUP BY
                coupon_code,
                LEFT(ord.dt, 7)
            ORDER BY 
                coupon_code,
                LEFT(ord.dt, 7) ASC
        ),
        preliminar_table AS (
            SELECT
                *,
                ROUND(
                    (
                        (monthly_sales 
                            - LAG(monthly_sales) OVER(PARTITION BY coupon_code ORDER BY year_month ASC)
                        )*100 / LAG(monthly_sales) OVER(PARTITION BY coupon_code ORDER BY year_month ASC)
                    ),
                    2) AS mom_growth,
                RANK() OVER(PARTITION BY year_month ORDER BY monthly_sales DESC) AS usage_rank,
                LAG(monthly_sales) OVER(PARTITION BY coupon_code ORDER BY year_month ASC) AS last_month__sales,
                LAG(monthly_sales, 2) OVER(PARTITION BY coupon_code ORDER BY year_month ASC) AS month_before_last_month__sales     
            FROM 
                monthly_table
            ORDER BY 
                year_month ASC
        ),
        -- for avg_last_3_months it is asked the avg of the last three months considering that, if a month is blank/null
        -- that avg is not by 3 (the count of months), but by the amount of months with data.
        -- There are several ways to solve this (as usual), this is a first functional approach that I made...
        final_table AS (   
            SELECT 
                coupon_code, year_month, monthly_sales, mom_growth, usage_rank,
                COALESCE(monthly_sales, 0) + COALESCE(last_month__sales, 0) + COALESCE(month_before_last_month__sales, 0) AS sum_last_3_months,
                CASE
                    WHEN monthly_sales> 0 THEN 1
                END monthly_sales__active,
                CASE
                    WHEN last_month__sales > 0 THEN 1
                END last_month__sales__active,
                CASE
                    WHEN month_before_last_month__sales > 0 THEN 1
                END month_before_last_month__sales__active
            FROM
                preliminar_table
        )
        
        SELECT
            coupon_code, year_month, monthly_sales, mom_growth, usage_rank,
            ROUND(
                sum_last_3_months / ( 
                                    COALESCE(monthly_sales__active, 0) + COALESCE(last_month__sales__active, 0) + COALESCE(month_before_last_month__sales__active, 0)
                                    ),
                2) AS avg_last_3_months
        FROM
            final_table
        ORDER BY 
            year_month,
            usage_rank
             '''
        ).df()

,coupon_code,year_month,monthly_sales,mom_growth,usage_rank,avg_last_3_months
0,DISCOUNT50,2022-08,79.72,NaN,1,79.72
1,DISCOUNT50,2022-09,63.34,-20.55,1,71.53
2,save20,2022-10,94.13,NaN,1,94.13
3,DISCOUNT50,2022-11,69.32,9.44,1,70.79
4,save20,2023-01,192.56,104.57,1,143.35
5,save20,2023-04,103.64,-46.18,1,130.11
6,COUPON123,2023-04,86.17,NaN,2,86.17
7,DISCOUNT50,2023-05,79.24,14.31,1,70.63
8,COUPON123,2023-08,76.09,-11.70,1,81.13


#### Result comparison

<img src="expected_output.png" style="display=block; margin:auto; width: 40%;">